the gola of this notebook is to pick the bestm odel only and use them instead of all the models. in other words the best 25% of the validation losses. will be used o determine the best model.

looking back at the psedo  inception models their losses are highr but their train/validation arocs are almost universally better. (with the exclusion of 450x450 which probably needed more training time).

so here's the plan:

1 - try an ensemble with psuedo labelled models (that worked well)

2 - if that dosent work ignore pseudo labelled models and just try to pick the best models.

3 - if it does work pick try to pick only the best pseudo and non pseudo labelled models. (also run rest of pseudo models)


In [4]:
import os, glob, bcolz, gc

import numpy as np
import pandas as pd

from tqdm import tqdm
from scipy import ndimage, misc
from scipy.stats import rankdata

from sklearn.externals import joblib
from sklearn.model_selection import train_test_split, KFold
from sklearn.metrics import roc_auc_score, log_loss, accuracy_score
from sklearn.preprocessing import MinMaxScaler
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor

from xgboost import XGBClassifier, XGBRegressor

import keras
from keras import backend as K
from keras import optimizers
from keras.models import Model, load_model

from keras.applications.inception_v3 import preprocess_input as preprocess_input_inceptionv3
from keras.applications.xception import preprocess_input as preprocess_input_xcep

from tensorflow.python.client import device_lib
device_lib.list_local_devices(), 'keras version: {}'.format(keras.__version__)

([name: "/cpu:0"
  device_type: "CPU"
  memory_limit: 268435456
  locality {
  }
  incarnation: 13042765116685983297, name: "/gpu:0"
  device_type: "GPU"
  memory_limit: 384565248
  locality {
    bus_id: 2
  }
  incarnation: 1828895668070095184
  physical_device_desc: "device: 0, name: Tesla K80, pci bus id: 0000:84:00.0"],
 'keras version: 2.0.6')

In [2]:
path = os.path.join('/scratch', 'yns207', 'data_invasive')
train_set = pd.read_csv(os.path.join(path, 'train_labels.csv'))
test_set = pd.read_csv(os.path.join(path, 'sample_submission.csv'))

def delete_model(model, clear_session=True):
    '''removes model!
    '''
    del model
    gc.collect()
    if clear_session: K.clear_session()

def read_img(img_path, img_shape):
    img = misc.imread(img_path)
    img = misc.imresize(img, img_shape)
    return img

def read_imgs(img_height, img_width):
    train_img, test_img = [],[]
    for img_path in tqdm(train_set['name'].iloc[:]):
        train_img.append(read_img(os.path.join(path, 'train', str(img_path)+'.jpg'), (img_height, img_width)))

    for img_path in tqdm(test_set['name'].iloc[:]):
        test_img.append(read_img(os.path.join(path, 'test', str(img_path)+'.jpg'), (img_height, img_width)))
    return np.array(train_img), np.array(test_img)

In [3]:
train_img_224, test_img_224 = read_imgs(224,224)
train_img_299, test_img_299 = read_imgs(299,299)
train_img_450, test_img_450 = read_imgs(450,450)
train_labels = train_set['invasive'].values

100%|██████████| 1531/1531 [00:52<00:00, 27.76it/s]


In [4]:
%cd $path

!cat results/invasive_incepv3_aug10_kfolds_299x299_history.out
!echo ' '
!cat results/invasive_incepv3_aug11*
!echo ' '
!cat results/invasive_xcep_aug11*
!echo ' '
!cat results/invasive_incepv3_aug13_pseudo*
!echo ' '
!cat results/invasive_xcep_aug13_pseudo*

/scratch/yns207/data_invasive
kfold: 0best model train loss: 0.027439485442031314best model valid loss: 0.0788394036157004best model train aroc score: 0.9984508394200023, valid aroc score: 0.9950655865783955
kfold: 1best model train loss: 0.03835649226339684best model valid loss: 0.03191655293139497best model train aroc score: 0.9991997013582065, valid aroc score: 0.9997786720321932
 
kfold: 0best model train loss: 0.062482393774851215best model valid loss: 0.08223816804376823best model train aroc score: 0.9990636884016482, valid aroc score: 0.9971810148346596
kfold: 1best model train loss: 0.01450507250596203best model valid loss: 0.04537784368660066best model train aroc score: 0.9999949175887007, valid aroc score: 0.9989280560709133
kfold: 2best model train loss: 0.019011034092547327best model valid loss: 0.05383957105167291best model train aroc score: 0.9999475842742594, valid aroc score: 0.9987525150905433
kfold: 3best model train loss: 0.02008771099969907best model valid loss: 0.0

In [10]:
model_files = [
    'invasive_incepv3_aug10_kfolds_299x299_0.model',
    'invasive_incepv3_aug10_kfolds_299x299_1.model',
    'invasive_xcep_aug11_kfolds_224x224_0.model',
    'invasive_xcep_aug11_kfolds_224x224_1.model',
    'invasive_xcep_aug11_kfolds_224x224_2.model',
    'invasive_xcep_aug11_kfolds_224x224_3.model',
    'invasive_xcep_aug11_kfolds_224x224_4.model',
    'invasive_xcep_aug11_kfolds_299x299_0.model',
    'invasive_xcep_aug11_kfolds_299x299_1.model',
    'invasive_xcep_aug11_kfolds_299x299_2.model',
    'invasive_xcep_aug11_kfolds_299x299_3.model',
    'invasive_xcep_aug11_kfolds_299x299_4.model',
    'invasive_xcep_aug13_pseudo_224x224_0.model',
    'invasive_xcep_aug13_pseudo_224x224_1.model',
    'invasive_xcep_aug13_pseudo_224x224_2.model',
    'invasive_xcep_aug13_pseudo_224x224_3.model',
    'invasive_xcep_aug13_pseudo_224x224_4.model',
    'invasive_incepv3_aug11_kfolds_224x224_0.model',
    'invasive_incepv3_aug11_kfolds_224x224_1.model',
    'invasive_incepv3_aug11_kfolds_224x224_2.model',
    'invasive_incepv3_aug11_kfolds_224x224_3.model',
    'invasive_incepv3_aug11_kfolds_224x224_4.model',
    'invasive_incepv3_aug11_kfolds_299x299_0.model',
    'invasive_incepv3_aug11_kfolds_299x299_1.model',
    'invasive_incepv3_aug11_kfolds_299x299_2.model',
    'invasive_incepv3_aug11_kfolds_299x299_3.model',
    'invasive_incepv3_aug11_kfolds_299x299_4.model',
    'invasive_incepv3_aug11_kfolds_450x450_0.model',
    'invasive_incepv3_aug11_kfolds_450x450_1.model',
    'invasive_incepv3_aug11_kfolds_450x450_2.model',
    'invasive_incepv3_aug11_kfolds_450x450_3.model',
    'invasive_incepv3_aug11_kfolds_450x450_4.model',
    'invasive_incepv3_aug13_pseudo_224x224_0.model',
    'invasive_incepv3_aug13_pseudo_224x224_1.model',
    'invasive_incepv3_aug13_pseudo_224x224_2.model',
    'invasive_incepv3_aug13_pseudo_224x224_3.model',
    'invasive_incepv3_aug13_pseudo_224x224_4.model',
    'invasive_incepv3_aug13_pseudo_299x299_0.model',
    'invasive_incepv3_aug13_pseudo_299x299_1.model',
    'invasive_incepv3_aug13_pseudo_299x299_2.model',
    'invasive_incepv3_aug13_pseudo_299x299_3.model',
    'invasive_incepv3_aug13_pseudo_299x299_4.model',
    'invasive_incepv3_aug13_pseudo_450x450_0.model',
    'invasive_incepv3_aug13_pseudo_450x450_1.model',
    'invasive_incepv3_aug13_pseudo_450x450_2.model'
]

In [11]:
preds_df = pd.DataFrame([], columns=['name'])
preds_df['name'] = test_set['name']

for model_name in model_files:
    print(model_name)
    model = load_model(os.path.join(path,model_name))
    if '224' in model_name:
        proc_test_img = preprocess_input_inceptionv3(test_img_224.astype(np.float32))
    elif '299' in model_name:
        proc_test_img = preprocess_input_inceptionv3(test_img_299.astype(np.float32))
    else:
        proc_test_img = preprocess_input_inceptionv3(test_img_450.astype(np.float32))
    preds_df[model_name] = pd.Series(model.predict(proc_test_img).flatten())
    delete_model(model)
preds_df.head()

invasive_incepv3_aug10_kfolds_299x299_0.model


/home/yns207/anaconda3/envs/keras-py3/lib/python3.5/site-packages/keras/models.py:281: UserWarning: Error in loading the saved optimizer state. As a result, your model is starting with a freshly initialized optimizer.
  warnings.warn('Error in loading the saved optimizer '


invasive_incepv3_aug10_kfolds_299x299_1.model
invasive_xcep_aug11_kfolds_224x224_0.model
invasive_xcep_aug11_kfolds_224x224_1.model
invasive_xcep_aug11_kfolds_224x224_2.model
invasive_xcep_aug11_kfolds_224x224_3.model
invasive_xcep_aug11_kfolds_224x224_4.model
invasive_xcep_aug11_kfolds_299x299_0.model
invasive_xcep_aug11_kfolds_299x299_1.model
invasive_xcep_aug11_kfolds_299x299_2.model
invasive_xcep_aug11_kfolds_299x299_3.model
invasive_xcep_aug11_kfolds_299x299_4.model
invasive_xcep_aug13_pseudo_224x224_0.model
invasive_xcep_aug13_pseudo_224x224_1.model
invasive_xcep_aug13_pseudo_224x224_2.model
invasive_xcep_aug13_pseudo_224x224_3.model
invasive_xcep_aug13_pseudo_224x224_4.model
invasive_incepv3_aug11_kfolds_224x224_0.model
invasive_incepv3_aug11_kfolds_224x224_1.model
invasive_incepv3_aug11_kfolds_224x224_2.model
invasive_incepv3_aug11_kfolds_224x224_3.model
invasive_incepv3_aug11_kfolds_224x224_4.model
invasive_incepv3_aug11_kfolds_299x299_0.model
invasive_incepv3_aug11_kfolds_299

,name,invasive_incepv3_aug10_kfolds_299x299_0.model,invasive_incepv3_aug10_kfolds_299x299_1.model,invasive_xcep_aug11_kfolds_224x224_0.model,invasive_xcep_aug11_kfolds_224x224_1.model,invasive_xcep_aug11_kfolds_224x224_2.model,invasive_xcep_aug11_kfolds_224x224_3.model,invasive_xcep_aug11_kfolds_224x224_4.model,invasive_xcep_aug11_kfolds_299x299_0.model,invasive_xcep_aug11_kfolds_299x299_1.model,...,invasive_incepv3_aug13_pseudo_224x224_3.model,invasive_incepv3_aug13_pseudo_224x224_4.model,invasive_incepv3_aug13_pseudo_299x299_0.model,invasive_incepv3_aug13_pseudo_299x299_1.model,invasive_incepv3_aug13_pseudo_299x299_2.model,invasive_incepv3_aug13_pseudo_299x299_3.model,invasive_incepv3_aug13_pseudo_299x299_4.model,invasive_incepv3_aug13_pseudo_450x450_0.model,invasive_incepv3_aug13_pseudo_450x450_1.model,invasive_incepv3_aug13_pseudo_450x450_2.model
0,1,0.999220,0.999937,0.999832,0.997136,0.998534,0.999998,0.999851,1.000000,1.000000,...,0.990677,0.996971,0.992153,0.997154,0.990139,0.991377,0.988569,0.989716,0.964873,0.998029
1,2,0.006064,0.018489,0.292350,0.021871,0.045619,0.011755,0.049390,0.087101,0.000298,...,0.032573,0.046906,0.010195,0.025843,0.056894,0.051597,0.026488,0.000974,0.030960,0.076614
2,3,0.011985,0.269732,0.047080,0.077081,0.053459,0.055718,0.004568,0.012156,0.014143,...,0.016454,0.090059,0.051287,0.097718,0.122369,0.107176,0.085269,0.044325,0.066671,0.100646
3,4,0.017386,0.077561,0.000072,0.024709,0.000225,0.000183,0.023354,0.000373,0.005312,...,0.019637,0.034847,0.017609,0.025839,0.017977,0.035340,0.034613,0.012286,0.133649,0.202498
4,5,0.978103,0.991908,0.975208,0.998245,0.997167,0.989112,0.996995,0.999627,0.999979,...,0.980899,0.987864,0.971801,0.988640,0.980422,0.966536,0.960517,0.990939,0.994033,0.988000


In [13]:
subm = pd.DataFrame([], columns=['name', 'invasive'])

subm['invasive'] = preds_df[[c for c in preds_df.columns if not c == 'name']].mean(axis=1)
subm['name'] = preds_df['name']

subm.head()

,name,invasive
0,1,0.993880
1,2,0.034207
2,3,0.079489
3,4,0.041603
4,5,0.979234


In [15]:
subm.to_csv(os.path.join(path,'results','subm_aug14_0.gz'), index=False, compression='gzip')

wow that did really well (0.99507LB or 9th place), maybe i should try to train those pseudo xception models or maybe just ft the ones i already have), lets now pick the best non pseudo models and the best pseudo models (top 25%) and only ensemble those. im comparing pseudo models and non pseudo models separately and ill take the top 25% of each because the loss functions need to be interpreted a little differently.

in other words pseudo labelling increased the loss on the train set but made the models generalize better. that's good.

the only thing were missing in pseudo labelling is the 299 of xcpetion net, so we can just load each model ant finetune for 20 epochs, and see what that gets us. then well have everything (excpet xception net 450x450 but that was too big).

In [5]:
model_files = [
    'invasive_incepv3_aug10_kfolds_299x299_1.model',
    'invasive_xcep_aug11_kfolds_299x299_1.model',
    'invasive_xcep_aug11_kfolds_299x299_2.model',
    'invasive_xcep_aug11_kfolds_299x299_4.model',
    'invasive_incepv3_aug11_kfolds_224x224_1.model',
    'invasive_incepv3_aug11_kfolds_224x224_2.model',
    'invasive_incepv3_aug11_kfolds_450x450_0.model',
    'invasive_incepv3_aug11_kfolds_450x450_3.model',
    'invasive_incepv3_aug13_pseudo_224x224_1.model',
    'invasive_incepv3_aug13_pseudo_224x224_3.model',
    'invasive_incepv3_aug13_pseudo_224x224_4.model',
    'invasive_incepv3_aug13_pseudo_299x299_1.model',
    'invasive_incepv3_aug13_pseudo_299x299_4.model',
]

In [6]:
preds_df = pd.DataFrame([], columns=['name'])
preds_df['name'] = test_set['name']

for model_name in model_files:
    print(model_name)
    model = load_model(os.path.join(path,model_name))
    if '224' in model_name:
        proc_test_img = preprocess_input_inceptionv3(test_img_224.astype(np.float32))
    elif '299' in model_name:
        proc_test_img = preprocess_input_inceptionv3(test_img_299.astype(np.float32))
    else:
        proc_test_img = preprocess_input_inceptionv3(test_img_450.astype(np.float32))
    preds_df[model_name] = pd.Series(model.predict(proc_test_img).flatten())
    delete_model(model)
preds_df.head()

invasive_incepv3_aug10_kfolds_299x299_1.model


/home/yns207/anaconda3/envs/keras-py3/lib/python3.5/site-packages/keras/models.py:281: UserWarning: Error in loading the saved optimizer state. As a result, your model is starting with a freshly initialized optimizer.
  warnings.warn('Error in loading the saved optimizer '


invasive_xcep_aug11_kfolds_299x299_1.model
invasive_xcep_aug11_kfolds_299x299_2.model
invasive_xcep_aug11_kfolds_299x299_4.model
invasive_incepv3_aug11_kfolds_224x224_1.model
invasive_incepv3_aug11_kfolds_224x224_2.model
invasive_incepv3_aug11_kfolds_450x450_0.model
invasive_incepv3_aug11_kfolds_450x450_3.model
invasive_incepv3_aug13_pseudo_224x224_1.model
invasive_incepv3_aug13_pseudo_224x224_3.model
invasive_incepv3_aug13_pseudo_224x224_4.model
invasive_incepv3_aug13_pseudo_299x299_1.model
invasive_incepv3_aug13_pseudo_299x299_4.model


,name,invasive_incepv3_aug10_kfolds_299x299_1.model,invasive_xcep_aug11_kfolds_299x299_1.model,invasive_xcep_aug11_kfolds_299x299_2.model,invasive_xcep_aug11_kfolds_299x299_4.model,invasive_incepv3_aug11_kfolds_224x224_1.model,invasive_incepv3_aug11_kfolds_224x224_2.model,invasive_incepv3_aug11_kfolds_450x450_0.model,invasive_incepv3_aug11_kfolds_450x450_3.model,invasive_incepv3_aug13_pseudo_224x224_1.model,invasive_incepv3_aug13_pseudo_224x224_3.model,invasive_incepv3_aug13_pseudo_224x224_4.model,invasive_incepv3_aug13_pseudo_299x299_1.model,invasive_incepv3_aug13_pseudo_299x299_4.model
0,1,0.999937,1.000000,0.999998,1.000000,0.995379,0.975407,0.996039,0.999967,0.990820,0.990677,0.996971,0.997154,0.988569
1,2,0.018489,0.000298,0.003632,0.011816,0.004982,0.003567,0.005403,0.019659,0.034217,0.032573,0.046906,0.025843,0.026488
2,3,0.269732,0.014143,0.000052,0.007512,0.139530,0.017842,0.005325,0.011113,0.084924,0.016454,0.090059,0.097718,0.085269
3,4,0.077561,0.005312,0.000028,0.000032,0.012415,0.001499,0.023607,0.072977,0.088001,0.019637,0.034847,0.025839,0.034613
4,5,0.991908,0.999979,0.999182,0.999819,0.997395,0.978214,0.993084,0.999270,0.991267,0.980899,0.987864,0.988640,0.960517


In [27]:
subm = pd.DataFrame([], columns=['name', 'invasive'])
subm['invasive'] = preds_df[[c for c in preds_df.columns if not c == 'name']].mean(axis=1)
subm['name'] = preds_df['name']
subm.head()

,name,invasive
0,1,0.994686
1,2,0.017990
2,3,0.064590
3,4,0.030490
4,5,0.989849


In [28]:
subm.to_csv(os.path.join(path,'results','subm_aug14_1.gz'), index=False, compression='gzip')

ok well see how the 'best modesl only' do, probably worse... they probably overfit somehow. NOPE it does better. 0.99513 LB which is an improvement but a small one. so what we want to do now is take the best of all the model normal and pseudo and combine them.

In [ ]:
# 18 total pseudo models, pick 5
# 27 total kfolds models, pick 7


# kfolds:
    
# 'results/invasive_incepv3_aug10_kfolds_299x299_history.out'

# 299 kfolds - 1

# 'results/invasive_incepv3_aug11*'
# 224 kfolds - 1, 2
# 450 kfolds - 0, 3

# 'results/invasive_xcep_aug11*'

# 299 - kfolds 1,2,4


# 8 total (fudged it a bit to add an extra model).

# 'results/invasive_incepv3_aug13_pseudo*'

# 224 - kfolds - 1,3,4
# 299 - kfolds- 1,4


# 13 t0tal model, these had the lowest vladiation losses.


# 'results/invasive_incepv3_aug10_kfolds_299x299_history.out'
# kfold: 0best model train loss: 0.027439485442031314best model valid loss: 0.0788394036157004best model train aroc score: 0.9984508394200023, valid aroc score: 0.9950655865783955
# kfold: 1best model train loss: 0.03835649226339684best model valid loss: 0.03191655293139497best model train aroc score: 0.9991997013582065, valid aroc score: 0.9997786720321932
 
# 'results/invasive_incepv3_aug11*'
# 224
# kfold: 0best model train loss: 0.062482393774851215best model valid loss: 0.08223816804376823best model train aroc score: 0.9990636884016482, valid aroc score: 0.9971810148346596
# kfold: 1best model train loss: 0.01450507250596203best model valid loss: 0.04537784368660066best model train aroc score: 0.9999949175887007, valid aroc score: 0.9989280560709133
# kfold: 2best model train loss: 0.019011034092547327best model valid loss: 0.05383957105167291best model train aroc score: 0.9999475842742594, valid aroc score: 0.9987525150905433
# kfold: 3best model train loss: 0.02008771099969907best model valid loss: 0.0721572656752562best model train aroc score: 0.9999732362360971, valid aroc score: 0.9957866883777733
# kfold: 4best model train loss: 0.046838037994302695best model valid loss: 0.09114888841631236best model train aroc score: 0.9998050198525241, valid aroc score: 0.9956603773584907

# 299
# kfold: 0best model train loss: 0.042133500170538886best model valid loss: 0.07393368971503637best model train aroc score: 0.9993607838324333, valid aroc score: 0.9963179074446681
# kfold: 1best model train loss: 0.1181186609992794best model valid loss: 0.12730378435198259best model train aroc score: 0.996424199143032, valid aroc score: 0.9970963287849253
# kfold: 2best model train loss: 0.0724632786769493best model valid loss: 0.111358704775864best model train aroc score: 0.9977780528719835, valid aroc score: 0.9912563667232598
# kfold: 3best model train loss: 0.07112650141477066best model valid loss: 0.09121641855938502best model train aroc score: 0.9982896680401151, valid aroc score: 0.9976796254834113
# kfold: 4best model train loss: 0.07328939590448907best model valid loss: 0.07524938396054415best model train aroc score: 0.9963285852260735, valid aroc score: 0.9967274333440411

# 450
# kfold: 0best model train loss: 0.02980839204213588best model valid loss: 0.050965858761359145best model train aroc score: 0.9998433647277578, valid aroc score: 0.9993421052631579
# kfold: 1best model train loss: 0.06426877763795956best model valid loss: 0.06793964136399994best model train aroc score: 0.9998695150393418, valid aroc score: 0.9999163879598663
# kfold: 2best model train loss: 0.030656124613286363best model valid loss: 0.06206890541354036best model train aroc score: 1.0, valid aroc score: 0.9962490023942537
# kfold: 3best model train loss: 0.018317573980787417best model valid loss: 0.05294950532854772best model train aroc score: 0.9999464724721944, valid aroc score: 0.9968451048239364
# kfold: 4best model train loss: 0.030412637890359154best model valid loss: 0.061787270194158965best model train aroc score: 0.9988088017599407, valid aroc score: 0.9959808347192523
 
# 'results/invasive_xcep_aug11*'
# 224:
# kfold: 0best model train loss: 0.03048578031109504best model valid loss: 0.1072594022341803best model train aroc score: 0.9994820024198428, valid aroc score: 0.9917218543046358
# kfold: 1best model train loss: 0.06364787272573297best model valid loss: 0.06237257628910617best model train aroc score: 0.9979475706803765, valid aroc score: 0.9957946086883385
# kfold: 2best model train loss: 0.041842009799153194best model valid loss: 0.0869028093110697best model train aroc score: 0.998658924480445, valid aroc score: 0.9946881287726358
# kfold: 3best model train loss: 0.033644665466836594best model valid loss: 0.088567715947778best model train aroc score: 0.998916864509949, valid aroc score: 0.9945216200352182
# kfold: 4best model train loss: 0.03989389581780288best model valid loss: 0.11687760636177977best model train aroc score: 0.9987512576801483, valid aroc score: 0.9897530215449292

# 299:
# kfold: 0best model train loss: 0.017340923531674872best model valid loss: 0.10297379009669123best model train aroc score: 0.9996728343910425, valid aroc score: 0.9936544991511036
# kfold: 1best model train loss: 0.012529143207343316best model valid loss: 0.04231889943606453best model train aroc score: 0.9999303235646971, valid aroc score: 0.998494983277592
# kfold: 2best model train loss: 0.013113725860450978best model valid loss: 0.049073818859021325best model train aroc score: 0.9999679166345833, valid aroc score: 0.9986287213323265
# kfold: 3best model train loss: 0.027738979974992913best model valid loss: 0.08526225128334852best model train aroc score: 0.9992809757197934, valid aroc score: 0.9952114924181963
# kfold: 4best model train loss: 0.01999965657332703best model valid loss: 0.045038245165490286best model train aroc score: 0.9997103391074106, valid aroc score: 0.9975083056478405
 
# 'results/invasive_incepv3_aug13_pseudo*'
# 224
# kfold: 0best model train loss: 0.07404465953898585best model valid loss: 0.09776425630087641best model train aroc score: 0.9999943779869648, valid aroc score: 0.9991063080899741
# kfold: 1best model train loss: 0.073031544911729best model valid loss: 0.0904598322958728best model train aroc score: 0.9999493631003287, valid aroc score: 0.999704783874334
# kfold: 2best model train loss: 0.07461232828796008best model valid loss: 0.10407043414957383best model train aroc score: 0.9999771559601393, valid aroc score: 0.9977395769381034
# kfold: 3best model train loss: 0.07570112979770369best model valid loss: 0.0892870456954233best model train aroc score: 0.9999724182870204, valid aroc score: 0.9994693700864369
# kfold: 4best model train loss: 0.07276285909011605best model valid loss: 0.09489505103990144best model train aroc score: 0.9999822768613619, valid aroc score: 0.9998071412827859

# 299
# kfold: 0best model train loss: 0.07920620365470063best model valid loss: 0.09685821586115864best model train aroc score: 0.999938718720654, valid aroc score: 0.9995977501751173
# kfold: 1best model train loss: 0.07763864777371292best model valid loss: 0.09146123688205395best model train aroc score: 0.9999154570521825, valid aroc score: 0.9996851385390428
# kfold: 2best model train loss: 0.07231500709526135best model valid loss: 0.09839534490716223best model train aroc score: 0.9999787662589789, valid aroc score: 0.9996986301369863
# kfold: 3best model train loss: 0.07681971980695124best model valid loss: 0.09849747092116112best model train aroc score: 0.999984075912278, valid aroc score: 0.9989568943644104
# kfold: 4best model train loss: 0.07837117731094127best model valid loss: 0.0913458882983214best model train aroc score: 0.9999879330005714, valid aroc score: 0.998491893929873

# 450
# kfold: 0best model train loss: 0.1373593466539009best model valid loss: 0.11933890568680926best model train aroc score: 0.9976590739125015, valid aroc score: 0.9991563149483157
# kfold: 1best model train loss: 0.10921669844385846best model valid loss: 0.11950838686593997best model train aroc score: 0.9988930867289934, valid aroc score: 0.99887551081978
# kfold: 2best model train loss: 0.14201612631330612best model valid loss: 0.14724062642240837best model train aroc score: 0.9979299771986209, valid aroc score: 0.9974457831325301

# 'results/invasive_xcep_aug13_pseudo*'
# 224
# kfold: 0best model train loss: 0.08194982705552593best model valid loss: 0.12476617023932404best model train aroc score: 0.9999602643285967, valid aroc score: 0.9967669063853599
# kfold: 1best model train loss: 0.08626303998513067best model valid loss: 0.10610095944279939best model train aroc score: 0.9998925623828542, valid aroc score: 0.9989801864801865
# kfold: 2best model train loss: 0.08593105301577372best model valid loss: 0.10297341792801626best model train aroc score: 0.9999396257139259, valid aroc score: 0.9991791646957344
# kfold: 3best model train loss: 0.08466251206647256best model valid loss: 0.10892751688271567best model train aroc score: 0.9998293027167229, valid aroc score: 0.9993657184617293
# kfold: 4best model train loss: 0.08594878955979396best model valid loss: 0.11140533374025931best model train aroc score: 0.9999208230251835, valid aroc score: 0.9979319821367227         

now well just try a rank average since i have a submission to use by 8pm.

In [7]:
for column in preds_df.columns:
    preds_df["rank_"+column] = rankdata(preds_df[column])
preds_df['r_avg'] = preds_df[[c for c in preds_df.columns if 'rank_' in c]].mean(axis=1)
preds_df.head()

,name,invasive_incepv3_aug10_kfolds_299x299_1.model,invasive_xcep_aug11_kfolds_299x299_1.model,invasive_xcep_aug11_kfolds_299x299_2.model,invasive_xcep_aug11_kfolds_299x299_4.model,invasive_incepv3_aug11_kfolds_224x224_1.model,invasive_incepv3_aug11_kfolds_224x224_2.model,invasive_incepv3_aug11_kfolds_450x450_0.model,invasive_incepv3_aug11_kfolds_450x450_3.model,invasive_incepv3_aug13_pseudo_224x224_1.model,...,rank_invasive_incepv3_aug11_kfolds_224x224_1.model,rank_invasive_incepv3_aug11_kfolds_224x224_2.model,rank_invasive_incepv3_aug11_kfolds_450x450_0.model,rank_invasive_incepv3_aug11_kfolds_450x450_3.model,rank_invasive_incepv3_aug13_pseudo_224x224_1.model,rank_invasive_incepv3_aug13_pseudo_224x224_3.model,rank_invasive_incepv3_aug13_pseudo_224x224_4.model,rank_invasive_incepv3_aug13_pseudo_299x299_1.model,rank_invasive_incepv3_aug13_pseudo_299x299_4.model,r_avg
0,1,0.999937,1.000000,0.999998,1.000000,0.995379,0.975407,0.996039,0.999967,0.990820,...,1290.0,1044.0,1165.0,1307.0,1029.0,1052.0,1218.0,1254.0,1054.0,1103.214286
1,2,0.018489,0.000298,0.003632,0.011816,0.004982,0.003567,0.005403,0.019659,0.034217,...,214.0,388.0,105.0,620.0,436.0,475.0,499.0,417.0,297.0,407.142857
2,3,0.269732,0.014143,0.000052,0.007512,0.139530,0.017842,0.005325,0.011113,0.084924,...,817.0,725.0,100.0,481.0,726.0,269.0,706.0,774.0,693.0,557.285714
3,4,0.077561,0.005312,0.000028,0.000032,0.012415,0.001499,0.023607,0.072977,0.088001,...,526.0,161.0,548.0,779.0,733.0,311.0,414.0,416.0,386.0,434.857143
4,5,0.991908,0.999979,0.999182,0.999819,0.997395,0.978214,0.993084,0.999270,0.991267,...,1381.0,1064.0,1097.0,1067.0,1036.0,1000.0,1007.0,1073.0,934.0,980.928571


In [25]:
subm = pd.DataFrame([], columns=['name', 'invasive'])
subm['name'] = preds_df['name']
subm['invasive'] =  MinMaxScaler(feature_range=(0.01,0.99)).fit_transform(preds_df['r_avg'].values.reshape(-1, 1))
subm.head()

,name,invasive
0,1,0.739748
1,2,0.235465
2,3,0.344239
3,4,0.255543
4,5,0.651155


In [26]:
subm.to_csv(os.path.join(path,'results','subm_aug14_2.gz'), index=False, compression='gzip')

ok taht wasnt better: 0.99441, rank avg seems to ruin the ordering put out by the model.

# adding some more models

plan:

add extra pseudo labelled models to ensemble if they are good (have a decent validation loss / roc).

In [3]:
train_img_299, test_img_299 = read_imgs(299,299)
train_labels = train_set['invasive'].values

100%|██████████| 1531/1531 [01:38<00:00, 14.73it/s]


In [9]:
model_files = [
'invasive_xcep_aug14_pseudo_ft_299x299_0.model',
'invasive_xcep_aug14_pseudo_ft_299x299_1.model',
'invasive_xcep_aug14_pseudo_ft_299x299_2.model',
'invasive_xcep_aug14_pseudo_ft_299x299_3.model',
'invasive_xcep_aug14_pseudo_ft_299x299_4.model'
]

preds_df = pd.DataFrame([], columns=['name'])
preds_df['name'] = train_labels

for model_name in model_files:
    print(model_name)
    model = load_model(os.path.join(path,model_name))
    proc_test_img = preprocess_input_xcep(train_img_299.astype(np.float32))
    preds_df[model_name] = pd.Series(model.predict(proc_test_img).flatten())
    delete_model(model)
preds_df.head()

invasive_xcep_aug14_pseudo_ft_299x299_0.model
invasive_xcep_aug14_pseudo_ft_299x299_1.model
invasive_xcep_aug14_pseudo_ft_299x299_2.model
invasive_xcep_aug14_pseudo_ft_299x299_3.model
invasive_xcep_aug14_pseudo_ft_299x299_4.model


,name,invasive_xcep_aug14_pseudo_ft_299x299_0.model,invasive_xcep_aug14_pseudo_ft_299x299_1.model,invasive_xcep_aug14_pseudo_ft_299x299_2.model,invasive_xcep_aug14_pseudo_ft_299x299_3.model,invasive_xcep_aug14_pseudo_ft_299x299_4.model
0,0,0.001030,0.002464,0.000297,0.000171,0.022697
1,0,0.077401,0.012859,0.001814,0.018453,0.022017
2,1,0.999912,0.994753,0.999962,0.999442,0.999998
3,0,0.000117,0.000052,0.000026,0.000029,0.000257
4,1,1.000000,1.000000,0.999983,1.000000,1.000000


In [25]:
model_0_preds = MinMaxScaler(feature_range=(0.000001,0.999999)).fit_transform(preds_df['invasive_xcep_aug14_pseudo_ft_299x299_0.model'].values.reshape(-1, 1))
model_1_preds = MinMaxScaler(feature_range=(0.000001,0.999999)).fit_transform(preds_df['invasive_xcep_aug14_pseudo_ft_299x299_1.model'].values.reshape(-1, 1))
model_2_preds = MinMaxScaler(feature_range=(0.000001,0.999999)).fit_transform(preds_df['invasive_xcep_aug14_pseudo_ft_299x299_2.model'].values.reshape(-1, 1))
model_3_preds = MinMaxScaler(feature_range=(0.000001,0.999999)).fit_transform(preds_df['invasive_xcep_aug14_pseudo_ft_299x299_3.model'].values.reshape(-1, 1))
model_4_preds = MinMaxScaler(feature_range=(0.000001,0.999999)).fit_transform(preds_df['invasive_xcep_aug14_pseudo_ft_299x299_4.model'].values.reshape(-1, 1))

In [26]:
log_loss(train_labels, model_0_preds), log_loss(train_labels, model_1_preds), log_loss(train_labels, model_2_preds), log_loss(train_labels, model_3_preds), log_loss(train_labels, model_4_preds)

(0.038350530377984964,
 0.017851822535711558,
 0.024859773425598083,
 0.035562479553843042,
 0.031183659575526889)

In [28]:
roc_auc_score(train_labels, model_0_preds), roc_auc_score(train_labels, model_1_preds), roc_auc_score(train_labels, model_2_preds), roc_auc_score(train_labels, model_3_preds), roc_auc_score(train_labels, model_4_preds)

(0.99919198079670202,
 0.99966733417260789,
 0.99968853346552999,
 0.99908027683015133,
 0.99882833138734695)

In [31]:
accuracy_score(train_labels, np.around(model_0_preds)), accuracy_score(train_labels, np.around(model_1_preds)), accuracy_score(train_labels, np.around(model_2_preds)), accuracy_score(train_labels, np.around(model_3_preds)), accuracy_score(train_labels, np.around(model_4_preds))

(0.98692810457516345,
 0.99607843137254903,
 0.9930283224400871,
 0.9882352941176471,
 0.99346405228758172)

1 and 2 have highest aroc and lowest losses we shoud add those. remember these were validated on this data so this is effectively theri validation accuracy (even though they were already trained on it. this is a measure of how well their low loss was preserved throught training on pseudo labels. (really wed need their riginal loss as well but this is fine).

# training metrics for new ensemble

In [36]:
train_img_224, test_img_224 = read_imgs(224,224)
train_img_299, test_img_299 = read_imgs(299,299)
train_img_450, test_img_450 = read_imgs(450,450)
train_labels = train_set['invasive'].values

100%|██████████| 1531/1531 [00:52<00:00, 29.24it/s]


In [41]:
model_files = [
    'invasive_incepv3_aug10_kfolds_299x299_1.model',
    'invasive_xcep_aug11_kfolds_299x299_1.model',
    'invasive_xcep_aug11_kfolds_299x299_2.model',
    'invasive_xcep_aug11_kfolds_299x299_4.model',
    'invasive_xcep_aug14_pseudo_ft_299x299_1.model',
    'invasive_xcep_aug14_pseudo_ft_299x299_2.model',
    'invasive_incepv3_aug11_kfolds_224x224_1.model',
    'invasive_incepv3_aug11_kfolds_224x224_2.model',
    'invasive_incepv3_aug11_kfolds_450x450_0.model',
    'invasive_incepv3_aug11_kfolds_450x450_3.model',
    'invasive_incepv3_aug13_pseudo_224x224_1.model',
    'invasive_incepv3_aug13_pseudo_224x224_3.model',
    'invasive_incepv3_aug13_pseudo_224x224_4.model',
    'invasive_incepv3_aug13_pseudo_299x299_1.model',
    'invasive_incepv3_aug13_pseudo_299x299_4.model',
]

preds_df_train = pd.DataFrame([], columns=['name'])
preds_df_train['name'] = train_labels

for model_name in model_files:
    print(model_name)
    model = load_model(os.path.join(path,model_name))
    if '224' in model_name:
        proc_img = preprocess_input_inceptionv3(train_img_224.astype(np.float32))
    elif '299' in model_name:
        proc_img = preprocess_input_inceptionv3(train_img_299.astype(np.float32))
    else:
        proc_img = preprocess_input_inceptionv3(train_img_450.astype(np.float32))
    preds_df_train[model_name] = pd.Series(model.predict(proc_img).flatten())
    delete_model(model)
preds_df_train.head()

invasive_incepv3_aug10_kfolds_299x299_1.model


/home/yns207/anaconda3/envs/keras-py3/lib/python3.5/site-packages/keras/models.py:281: UserWarning: Error in loading the saved optimizer state. As a result, your model is starting with a freshly initialized optimizer.
  warnings.warn('Error in loading the saved optimizer '


invasive_xcep_aug11_kfolds_299x299_1.model
invasive_xcep_aug11_kfolds_299x299_2.model
invasive_xcep_aug11_kfolds_299x299_4.model
invasive_xcep_aug14_pseudo_ft_299x299_1.model
invasive_xcep_aug14_pseudo_ft_299x299_2.model
invasive_incepv3_aug11_kfolds_224x224_1.model
invasive_incepv3_aug11_kfolds_224x224_2.model
invasive_incepv3_aug11_kfolds_450x450_0.model
invasive_incepv3_aug11_kfolds_450x450_3.model
invasive_incepv3_aug13_pseudo_224x224_1.model
invasive_incepv3_aug13_pseudo_224x224_3.model
invasive_incepv3_aug13_pseudo_224x224_4.model
invasive_incepv3_aug13_pseudo_299x299_1.model
invasive_incepv3_aug13_pseudo_299x299_4.model


,name,invasive_incepv3_aug10_kfolds_299x299_1.model,invasive_xcep_aug11_kfolds_299x299_1.model,invasive_xcep_aug11_kfolds_299x299_2.model,invasive_xcep_aug11_kfolds_299x299_4.model,invasive_xcep_aug14_pseudo_ft_299x299_1.model,invasive_xcep_aug14_pseudo_ft_299x299_2.model,invasive_incepv3_aug11_kfolds_224x224_1.model,invasive_incepv3_aug11_kfolds_224x224_2.model,invasive_incepv3_aug11_kfolds_450x450_0.model,invasive_incepv3_aug11_kfolds_450x450_3.model,invasive_incepv3_aug13_pseudo_224x224_1.model,invasive_incepv3_aug13_pseudo_224x224_3.model,invasive_incepv3_aug13_pseudo_224x224_4.model,invasive_incepv3_aug13_pseudo_299x299_1.model,invasive_incepv3_aug13_pseudo_299x299_4.model
0,0,0.011497,0.000081,2.598423e-05,1.556902e-03,0.002464,0.000297,0.003617,0.002008,0.006519,0.006436,0.003418,0.003613,0.020670,0.003738,0.004922
1,0,0.000088,0.001395,2.559908e-04,7.384096e-02,0.012859,0.001814,0.021587,0.000337,0.071751,0.008619,0.015926,0.013897,0.000691,0.024177,0.013948
2,1,0.999820,0.995244,9.993926e-01,9.999986e-01,0.994753,0.999962,0.993694,0.987813,0.990071,0.999852,0.998253,0.999630,0.996576,0.999223,0.997008
3,0,0.007591,0.000002,1.312236e-07,5.533665e-10,0.000052,0.000026,0.093534,0.007037,0.005201,0.046485,0.006873,0.036359,0.019254,0.002051,0.025521
4,1,1.000000,1.000000,9.999477e-01,1.000000e+00,1.000000,0.999983,0.999607,0.999682,0.995446,0.999997,0.998713,0.999457,0.998453,0.998412,0.999273


In [61]:
for col in preds_df_train.columns:
    train_preds = pd.DataFrame([], columns=['name', 'invasive'])
    if col == 'name':
        col = 'with all'
        exclusion_list = []
    else:
        exclusion_list = [col]
    train_preds['invasive'] = preds_df_train[[c for c in preds_df_train.columns if not c in exclusion_list]].mean(axis=1)
    train_preds['name'] =  preds_df_train['name']
    
    print('without {}'.format(col))
    print(log_loss(train_labels, train_preds['invasive']), roc_auc_score(train_labels, train_preds['invasive']), accuracy_score(train_labels, np.around(model_0_preds)))
    

without with all
0.0159226715819 1.0 0.986928104575
without invasive_incepv3_aug10_kfolds_299x299_1.model
0.0149633506681 1.0 0.986928104575
without invasive_xcep_aug11_kfolds_299x299_1.model
0.0161698736047 1.0 0.986928104575
without invasive_xcep_aug11_kfolds_299x299_2.model
0.0160944932504 1.0 0.986928104575
without invasive_xcep_aug11_kfolds_299x299_4.model
0.0161146919751 1.0 0.986928104575
without invasive_xcep_aug14_pseudo_ft_299x299_1.model
0.0160911444721 1.0 0.986928104575
without invasive_xcep_aug14_pseudo_ft_299x299_2.model
0.0159697590878 1.0 0.986928104575
without invasive_incepv3_aug11_kfolds_224x224_1.model
0.0158180491645 1.0 0.986928104575
without invasive_incepv3_aug11_kfolds_224x224_2.model
0.0156122910446 1.0 0.986928104575
without invasive_incepv3_aug11_kfolds_450x450_0.model
0.014981105533 1.0 0.986928104575
without invasive_incepv3_aug11_kfolds_450x450_3.model
0.0157577449534 1.0 0.986928104575
without invasive_incepv3_aug13_pseudo_224x224_1.model
0.016216050976

# new ensemble aug15 w/ pseudo xception finetune

In [ ]:
model_files = [
    'invasive_incepv3_aug10_kfolds_299x299_1.model',
    'invasive_xcep_aug11_kfolds_299x299_1.model',
    'invasive_xcep_aug11_kfolds_299x299_2.model',
    'invasive_xcep_aug11_kfolds_299x299_4.model',
    'invasive_xcep_aug14_pseudo_ft_299x299_1.model',
    'invasive_xcep_aug14_pseudo_ft_299x299_2.model',
    'invasive_incepv3_aug11_kfolds_224x224_1.model',
    'invasive_incepv3_aug11_kfolds_224x224_2.model',
    'invasive_incepv3_aug11_kfolds_450x450_0.model',
    'invasive_incepv3_aug11_kfolds_450x450_3.model',
    'invasive_incepv3_aug13_pseudo_224x224_1.model',
    'invasive_incepv3_aug13_pseudo_224x224_3.model',
    'invasive_incepv3_aug13_pseudo_224x224_4.model',
    'invasive_incepv3_aug13_pseudo_299x299_1.model',
    'invasive_incepv3_aug13_pseudo_299x299_4.model',
]

preds_df = pd.DataFrame([], columns=['name'])
preds_df['name'] = test_set['name']

for model_name in model_files:
    print(model_name)
    model = load_model(os.path.join(path,model_name))
    if '224' in model_name:
        proc_test_img = preprocess_input_inceptionv3(test_img_224.astype(np.float32))
    elif '299' in model_name:
        proc_test_img = preprocess_input_inceptionv3(test_img_299.astype(np.float32))
    else:
        proc_test_img = preprocess_input_inceptionv3(test_img_450.astype(np.float32))
    preds_df[model_name] = pd.Series(model.predict(proc_test_img).flatten())
    delete_model(model)
preds_df.head()

invasive_incepv3_aug10_kfolds_299x299_1.model


/home/yns207/anaconda3/envs/keras-py3/lib/python3.5/site-packages/keras/models.py:281: UserWarning: Error in loading the saved optimizer state. As a result, your model is starting with a freshly initialized optimizer.
  warnings.warn('Error in loading the saved optimizer '


In [33]:
preds_df.head()

,name,invasive_incepv3_aug10_kfolds_299x299_1.model,invasive_xcep_aug11_kfolds_299x299_1.model,invasive_xcep_aug11_kfolds_299x299_2.model,invasive_xcep_aug11_kfolds_299x299_4.model,invasive_xcep_aug14_pseudo_ft_299x299_1.model,invasive_xcep_aug14_pseudo_ft_299x299_2.model
0,1,0.999937,1.000000,0.999998,1.000000,0.999999,0.999999
1,2,0.018489,0.000298,0.003632,0.011816,0.000707,0.013593
2,3,0.269732,0.014143,0.000052,0.007512,0.016485,0.001794
3,4,0.077561,0.005312,0.000028,0.000032,0.000681,0.002819
4,5,0.991908,0.999979,0.999182,0.999819,0.999995,0.999847


In [44]:
subm = pd.DataFrame([], columns=['name', 'invasive'])

subm['invasive'] = preds_df[[c for c in preds_df.columns if not c == 'name']].mean(axis=1)
subm['name'] = preds_df['name']

subm.head()

,name,invasive
0,1,0.999989
1,2,0.008089
2,3,0.051620
3,4,0.014406
4,5,0.998455


In [45]:
subm.to_csv(os.path.join(path, 'results', 'subm_aug15_0.gz'), index=False, compression='gzip')

that didnt do that well (in fact much worse):
0.98959

# new attempt

lets make training predictions for all our models and see which ones work and which dont.

In [ ]:
model_files = [
    'invasive_incepv3_aug10_kfolds_299x299_0.model',
    'invasive_incepv3_aug10_kfolds_299x299_1.model',
    'invasive_xcep_aug11_kfolds_224x224_0.model',
    'invasive_xcep_aug11_kfolds_224x224_1.model',
    'invasive_xcep_aug11_kfolds_224x224_2.model',
    'invasive_xcep_aug11_kfolds_224x224_3.model',
    'invasive_xcep_aug11_kfolds_224x224_4.model',
    'invasive_xcep_aug11_kfolds_299x299_0.model',
    'invasive_xcep_aug11_kfolds_299x299_1.model',
    'invasive_xcep_aug11_kfolds_299x299_2.model',
    'invasive_xcep_aug11_kfolds_299x299_3.model',
    'invasive_xcep_aug11_kfolds_299x299_4.model',
    'invasive_xcep_aug13_pseudo_224x224_0.model',
    'invasive_xcep_aug13_pseudo_224x224_1.model',
    'invasive_xcep_aug13_pseudo_224x224_2.model',
    'invasive_xcep_aug13_pseudo_224x224_3.model',
    'invasive_xcep_aug13_pseudo_224x224_4.model',
    'invasive_xcep_aug14_pseudo_ft_299x299_0.model',
    'invasive_xcep_aug14_pseudo_ft_299x299_1.model',
    'invasive_xcep_aug14_pseudo_ft_299x299_2.model',
    'invasive_xcep_aug14_pseudo_ft_299x299_3.model',
    'invasive_xcep_aug14_pseudo_ft_299x299_4.model',
    'invasive_incepv3_aug11_kfolds_224x224_0.model',
    'invasive_incepv3_aug11_kfolds_224x224_1.model',
    'invasive_incepv3_aug11_kfolds_224x224_2.model',
    'invasive_incepv3_aug11_kfolds_224x224_3.model',
    'invasive_incepv3_aug11_kfolds_224x224_4.model',
    'invasive_incepv3_aug11_kfolds_299x299_0.model',
    'invasive_incepv3_aug11_kfolds_299x299_1.model',
    'invasive_incepv3_aug11_kfolds_299x299_2.model',
    'invasive_incepv3_aug11_kfolds_299x299_3.model',
    'invasive_incepv3_aug11_kfolds_299x299_4.model',
    'invasive_incepv3_aug11_kfolds_450x450_0.model',
    'invasive_incepv3_aug11_kfolds_450x450_1.model',
    'invasive_incepv3_aug11_kfolds_450x450_2.model',
    'invasive_incepv3_aug11_kfolds_450x450_3.model',
    'invasive_incepv3_aug11_kfolds_450x450_4.model',
    'invasive_incepv3_aug13_pseudo_224x224_0.model',
    'invasive_incepv3_aug13_pseudo_224x224_1.model',
    'invasive_incepv3_aug13_pseudo_224x224_2.model',
    'invasive_incepv3_aug13_pseudo_224x224_3.model',
    'invasive_incepv3_aug13_pseudo_224x224_4.model',
    'invasive_incepv3_aug13_pseudo_299x299_0.model',
    'invasive_incepv3_aug13_pseudo_299x299_1.model',
    'invasive_incepv3_aug13_pseudo_299x299_2.model',
    'invasive_incepv3_aug13_pseudo_299x299_3.model',
    'invasive_incepv3_aug13_pseudo_299x299_4.model',
    'invasive_incepv3_aug13_pseudo_450x450_0.model',
    'invasive_incepv3_aug13_pseudo_450x450_1.model',
    'invasive_incepv3_aug13_pseudo_450x450_2.model',
    'invasive_incepv3_aug13_pseudo_450x450_3.model',
    'invasive_incepv3_aug13_pseudo_450x450_4.model',
]

preds_df_train = pd.DataFrame([], columns=['name'])
preds_df_train['name'] = train_labels

for model_name in model_files:
    print(model_name)
    model = load_model(os.path.join(path,model_name))
    if '224' in model_name:
        proc_img = preprocess_input_inceptionv3(train_img_224.astype(np.float32))
    elif '299' in model_name:
        proc_img = preprocess_input_inceptionv3(train_img_299.astype(np.float32))
    else:
        proc_img = preprocess_input_inceptionv3(train_img_450.astype(np.float32))
    preds_df_train[model_name] = pd.Series(model.predict(proc_img).flatten())
    delete_model(model)
preds_df_train.head()